# 6.2 Defining Tables, Views, and Sinks in Lakeflow

This notebook provides hands-on implementation guidance for defining tables, materialized views, temporary views, and sinks using `pyspark.pipelines`. We'll explore configuration options, functional patterns, and best practices for each table type.

## Learning Objectives

By the end of this notebook, you will understand how to:
- Define batch tables with `@dp.table` and configuration options
- Create materialized views for pre-computed aggregations
- Use temporary views for intermediate transformations
- Configure table properties (partitioning, clustering, schema enforcement)
- Define custom sinks for specialized output patterns
- Apply functional programming principles to table definitions
- Structure table definitions for testability and reusability

## Prerequisites

- Completion of Notebook 6.1 (Introduction to pyspark.pipelines)
- Understanding of PySpark DataFrames
- Knowledge of Delta Lake table properties
- Familiarity with functional programming concepts

In [ ]:
# Platform setup detection
# In Databricks: Keep commented
# In Local: Uncomment this line
# %run 00_Environment_Setup.ipynb

In [ ]:
# Essential imports for Lakeflow pipelines
# NOTE: Actual pyspark.pipelines requires Spark 4.1+ and pipeline execution mode
# This notebook demonstrates concepts with educational examples

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import *
from typing import Dict, List, Optional
from dataclasses import dataclass

# In a real Lakeflow pipeline:
# from pyspark import pipelines as dp

print("✅ Imports complete - Ready for table definition demonstration!")

## 1. Defining Batch Tables with @dp.table

### Basic Table Definition

The `@dp.table` decorator defines a materialized Delta table that is fully refreshed on each pipeline run.

```python
from pyspark import pipelines as dp

@dp.table(
    name="customers",                    # Table name (optional, defaults to function name)
    comment="Customer master table"      # Documentation
)
def customers():
    """
    Pure function returning a DataFrame.
    No side effects - Lakeflow handles materialization.
    """
    return spark.read.table("raw.customers")
```

### Table Definition Best Practices

**✅ DO: Keep table functions pure**
```python
@dp.table
def clean_customers():
    """Pure function - only returns DataFrame"""
    return (
        spark.table("raw.customers")
        .filter(F.col("status") == "active")
        .select("customer_id", "name", "email", "country")
    )
    # ✓ No .write(), no .collect(), no side effects
```

**❌ DON'T: Include side effects in table functions**
```python
@dp.table
def bad_customers():
    df = spark.table("raw.customers")
    
    # ❌ WRONG: These are prohibited in Lakeflow
    df.write.save("/some/path")           # Side effect
    count = df.count()                    # Action (triggers computation)
    print(f"Processing {count} records") # Side effect
    df.cache()                            # State mutation
    
    return df
```

### Configuration Options

```python
@dp.table(
    name="customers",
    comment="Customer master table with comprehensive metadata",
    
    # Table properties
    table_properties={
        "delta.autoOptimize.optimizeWrite": "true",
        "delta.autoOptimize.autoCompact": "true",
        "delta.targetFileSize": "128mb"
    },
    
    # Partitioning (for large tables)
    partition_cols=["country", "signup_year"],
    
    # Path specification (optional)
    path="/mnt/delta/customers"
)
def customers():
    return (
        spark.table("raw.customers")
        .withColumn("signup_year", F.year("signup_date"))
    )
```

## 2. Materialized Views with @dp.materialized_view

### When to Use Materialized Views

Materialized views are ideal for:
- Complex joins across multiple tables
- Expensive aggregations used by multiple downstream consumers
- Pre-computed metrics for dashboards and reporting
- Denormalized tables for query performance

### Basic Materialized View

```python
@dp.materialized_view(
    name="customer_order_summary",
    comment="Pre-computed customer order metrics"
)
def customer_order_summary():
    """
    Expensive join and aggregation computed once per pipeline run.
    Results are materialized for fast query performance.
    """
    customers = dp.read("customers")
    orders = dp.read("orders")
    
    return (
        orders
        .join(customers, "customer_id")
        .groupBy(
            "customer_id",
            "customer_name",
            "country"
        )
        .agg(
            F.count("order_id").alias("total_orders"),
            F.sum("order_total").alias("total_spent"),
            F.avg("order_total").alias("avg_order_value"),
            F.max("order_date").alias("last_order_date")
        )
    )
```

### Functional Composition with Materialized Views

```python
# Pure transformation functions
def enrich_with_customer_tier(df: DataFrame) -> DataFrame:
    """Pure function: Classify customers by spending"""
    return df.withColumn(
        "tier",
        F.when(F.col("total_spent") >= 10000, "Platinum")
         .when(F.col("total_spent") >= 5000, "Gold")
         .when(F.col("total_spent") >= 1000, "Silver")
         .otherwise("Bronze")
    )

def calculate_customer_health_score(df: DataFrame) -> DataFrame:
    """Pure function: Calculate engagement score"""
    return df.withColumn(
        "health_score",
        (
            (F.col("total_orders") * 10) +
            (F.col("total_spent") / 100) +
            (F.datediff(F.current_date(), F.col("last_order_date")) * -1)
        )
    )

# Compose pure functions in materialized view
@dp.materialized_view(name="customer_analytics")
def customer_analytics():
    """Composed analytics from pure functions"""
    return (
        dp.read("customer_order_summary")
        .transform(enrich_with_customer_tier)
        .transform(calculate_customer_health_score)
    )
```

### Materialized View vs Table: Decision Matrix

| Criteria | @dp.table | @dp.materialized_view |
|----------|-----------|------------------------|
| **Purpose** | Primary data storage | Pre-computed aggregations |
| **Source** | Raw/external data | Derived from other tables |
| **Complexity** | Simple transformations | Complex joins/aggregations |
| **Consumers** | Few to many | Many consumers |
| **Update Pattern** | Replace or append | Refresh on pipeline run |
| **Best For** | Source tables, dimensions | Analytics, reporting, metrics |


## 3. Temporary Views with @dp.temporary_view

### Purpose and Benefits

Temporary views are **logical views** that:
- Are NOT materialized to storage
- Are computed on-demand when referenced
- Save storage costs for intermediate transformations
- Enable reusable subquery logic

### When to Use Temporary Views

**✅ Use temporary views for:**
- Intermediate filter steps
- Reusable subqueries used by multiple tables
- Logic that's referenced only within the same pipeline
- Simple transformations that are cheap to recompute

**❌ Don't use temporary views for:**
- Expensive computations (joins, aggregations)
- Tables queried by external systems
- Results that need to be persisted
- Complex transformations computed multiple times

### Basic Temporary View

```python
@dp.temporary_view(
    name="active_customers",
    comment="Logical view of active customers (not materialized)"
)
def active_customers():
    """Filter to active customers - no storage cost"""
    return (
        dp.read("customers")
        .filter(F.col("status") == "active")
        .filter(F.col("email_verified") == True)
    )

# Multiple tables can reference this view
@dp.table
def active_customer_orders():
    return (
        dp.read("active_customers")  # Computed on-demand
        .join(dp.read("orders"), "customer_id")
    )

@dp.table
def active_customer_subscriptions():
    return (
        dp.read("active_customers")  # Recomputed (not cached)
        .join(dp.read("subscriptions"), "customer_id")
    )
```

### Functional Reusability Pattern

```python
# Define reusable filters as temporary views
@dp.temporary_view
def high_value_customers():
    """Customers with significant lifetime value"""
    return (
        dp.read("customer_order_summary")
        .filter(F.col("total_spent") >= 5000)
        .filter(F.col("total_orders") >= 10)
    )

@dp.temporary_view
def recent_customers():
    """Customers who ordered in last 30 days"""
    return (
        dp.read("customer_order_summary")
        .filter(
            F.datediff(F.current_date(), F.col("last_order_date")) <= 30
        )
    )

# Combine views for specialized segments
@dp.table
def vip_active_customers():
    """High-value customers who are recently active"""
    return (
        dp.read("high_value_customers")
        .join(
            dp.read("recent_customers"),
            "customer_id",
            "inner"
        )
    )
```

### Performance Consideration

```python
# ⚠️ CAUTION: Expensive view used multiple times
@dp.temporary_view
def expensive_aggregation():
    """Complex computation - consider materializing!"""
    return (
        dp.read("large_table")
        .join(dp.read("another_large_table"), "key")  # Expensive
        .groupBy("dimension").agg(F.sum("metric"))     # Expensive
    )

# This will recompute the expensive aggregation THREE times:
@dp.table
def report_1():
    return dp.read("expensive_aggregation").filter(...)  # Compute #1

@dp.table
def report_2():
    return dp.read("expensive_aggregation").filter(...)  # Compute #2

@dp.table
def report_3():
    return dp.read("expensive_aggregation").filter(...)  # Compute #3

# ✅ BETTER: Materialize expensive computation
@dp.materialized_view  # Changed from temporary_view
def expensive_aggregation():
    """Computed once, reused three times"""
    return (...)  # Same logic, now materialized
```

## 4. Table Properties and Configuration

### Delta Lake Optimization Properties

```python
@dp.table(
    name="optimized_events",
    table_properties={
        # Auto-optimization
        "delta.autoOptimize.optimizeWrite": "true",
        "delta.autoOptimize.autoCompact": "true",
        
        # File sizing
        "delta.targetFileSize": "128mb",
        
        # Data skipping
        "delta.dataSkippingNumIndexedCols": "32",
        
        # Retention
        "delta.deletedFileRetentionDuration": "interval 7 days",
        "delta.logRetentionDuration": "interval 30 days"
    }
)
def optimized_events():
    return spark.table("raw.events")
```

### Partitioning Strategies

```python
# Time-based partitioning (common for event data)
@dp.table(
    name="events_partitioned",
    partition_cols=["event_date"],  # Partition by date
    comment="Events partitioned by date for efficient time-range queries"
)
def events_partitioned():
    return (
        spark.table("raw.events")
        .withColumn("event_date", F.to_date("event_timestamp"))
    )

# Multi-level partitioning (for very large tables)
@dp.table(
    name="transactions_partitioned",
    partition_cols=["year", "month", "country"],
    comment="Hierarchical partitioning for global transaction data"
)
def transactions_partitioned():
    return (
        spark.table("raw.transactions")
        .withColumn("year", F.year("transaction_date"))
        .withColumn("month", F.month("transaction_date"))
    )
```

### Liquid Clustering (Databricks-specific)

```python
@dp.table(
    name="customers_clustered",
    table_properties={
        "delta.enableChangeDataFeed": "true",
        "delta.feature.clustering": "supported"
    },
    cluster_cols=["country", "customer_tier"],  # Databricks-specific
    comment="Customers with liquid clustering for flexible query patterns"
)
def customers_clustered():
    return spark.table("raw.customers")
# Benefits: Better than static partitioning for evolving query patterns
```

## 5. Advanced Patterns: Dependency Management

### Reading from Other Pipeline Tables

```python
# Method 1: Using dp.read() (recommended)
@dp.table
def silver_customers():
    return dp.read("bronze_customers").filter(...)
    # ✓ Lakeflow tracks dependency: bronze → silver

# Method 2: Using spark.table() for external tables
@dp.table
def external_data():
    return spark.table("external_catalog.schema.table")
    # ✓ For tables outside the pipeline
```

### Complex Dependency Graphs

```python
# Bronze layer: Raw data ingestion
@dp.table
def bronze_orders():
    return spark.read.table("raw.orders")

@dp.table
def bronze_customers():
    return spark.read.table("raw.customers")

@dp.table
def bronze_products():
    return spark.read.table("raw.products")

# Silver layer: Cleaned and joined data
@dp.table
def silver_orders_enriched():
    """Orders enriched with customer and product data"""
    orders = dp.read("bronze_orders")
    customers = dp.read("bronze_customers")
    products = dp.read("bronze_products")
    
    return (
        orders
        .join(customers, "customer_id")
        .join(products, "product_id")
        .select(
            "order_id",
            "order_date",
            "customer_id",
            customers["customer_name"],
            "product_id",
            products["product_name"],
            "quantity",
            "unit_price",
            (F.col("quantity") * F.col("unit_price")).alias("total")
        )
    )
# Dependencies: bronze_orders, bronze_customers, bronze_products → silver_orders_enriched

# Gold layer: Business aggregations
@dp.materialized_view
def gold_daily_revenue():
    """Daily revenue aggregation"""
    return (
        dp.read("silver_orders_enriched")
        .groupBy(F.to_date("order_date").alias("date"))
        .agg(
            F.sum("total").alias("revenue"),
            F.count("order_id").alias("order_count"),
            F.countDistinct("customer_id").alias("unique_customers")
        )
    )
# Dependency: silver_orders_enriched → gold_daily_revenue

# Lakeflow execution order (automatically determined):
# 1. bronze_orders, bronze_customers, bronze_products (parallel)
# 2. silver_orders_enriched (waits for all bronze)
# 3. gold_daily_revenue (waits for silver)
```

### Cross-Pipeline References

```python
# Reading from another pipeline's output
@dp.table
def my_table():
    # Reference table from a different pipeline
    other_pipeline_table = spark.table("catalog.schema.other_pipeline_output")
    return other_pipeline_table.filter(...)
    # Note: Lakeflow Jobs can orchestrate cross-pipeline dependencies
```

## 6. Custom Sinks with dp.create_sink()

### What Are Sinks?

Sinks allow you to write pipeline data to custom destinations beyond Delta tables.

**Common Use Cases:**
- Write to external databases (PostgreSQL, MySQL, etc.)
- Export to cloud storage (S3, Azure Blob, GCS)
- Send to messaging systems (Kafka, Event Hubs)
- Push to APIs or web services

### Basic Sink Definition (Databricks-specific)

```python
# Create a custom sink for a table
dp.create_sink(
    source=dp.read("gold_customer_metrics"),
    destination="external_analytics",
    mode="overwrite",
    options={
        "url": "jdbc:postgresql://host:5432/analytics",
        "dbtable": "customer_metrics",
        "user": "etl_user",
        "password": dbutils.secrets.get("db", "password")
    }
)
```

### Sink Patterns

```python
# Pattern 1: Export to Parquet for external systems
dp.create_sink(
    source=dp.read("gold_daily_revenue"),
    destination="/mnt/exports/revenue",
    format="parquet",
    mode="overwrite",
    partition_by=["year", "month"]
)

# Pattern 2: Incremental export (append mode)
dp.create_sink(
    source=dp.read("silver_events"),
    destination="/mnt/exports/events",
    format="delta",
    mode="append"
)

# Pattern 3: Export to external warehouse
dp.create_sink(
    source=dp.read("gold_customer_analytics"),
    destination="analytics_warehouse",
    options={
        "url": "jdbc:snowflake://account.snowflakecomputing.com",
        "dbtable": "analytics.customer_metrics",
        "sfWarehouse": "COMPUTE_WH",
        "sfDatabase": "ANALYTICS",
        "sfSchema": "PUBLIC"
    }
)
```

### Sink vs Table: When to Use Each

| Feature | @dp.table | dp.create_sink() |
|---------|-----------|------------------|
| **Storage** | Delta Lake (lakehouse) | Custom destination |
| **Primary Use** | Pipeline transformations | External system integration |
| **Queryable in Pipeline** | Yes (via dp.read()) | No |
| **Unity Catalog** | Integrated | External |
| **Best For** | Internal pipeline data | Exporting to external systems |


## 7. Testable and Reusable Table Definitions

### Functional Composition for Testability

```python
# Pure transformation functions (testable in isolation)
def filter_active_status(df: DataFrame) -> DataFrame:
    """Pure function: Filter to active records"""
    return df.filter(F.col("status") == "active")

def enrich_with_age_group(df: DataFrame) -> DataFrame:
    """Pure function: Add age group classification"""
    return df.withColumn(
        "age_group",
        F.when(F.col("age") < 18, "Under 18")
         .when(F.col("age") < 35, "18-34")
         .when(F.col("age") < 55, "35-54")
         .otherwise("55+")
    )

def standardize_country_codes(df: DataFrame) -> DataFrame:
    """Pure function: Normalize country codes to ISO 3166-1 alpha-2"""
    return df.withColumn(
        "country_code",
        F.upper(F.trim(F.col("country")))
    )

# Compose pure functions in table definition
@dp.table(
    name="customers_processed",
    comment="Customers with derived attributes"
)
def customers_processed():
    """Table definition composed from testable pure functions"""
    return (
        dp.read("bronze_customers")
        .transform(filter_active_status)
        .transform(enrich_with_age_group)
        .transform(standardize_country_codes)
        .select(
            "customer_id",
            "name",
            "email",
            "age",
            "age_group",
            "country_code",
            "signup_date"
        )
    )
```

### Testing Pure Transformation Functions

```python
# test_transformations.py
import pytest
from pyspark.sql import SparkSession
from chispa.dataframe_comparer import assert_df_equality

def test_filter_active_status(spark: SparkSession):
    """Test active status filter in isolation"""
    # Input data
    input_df = spark.createDataFrame([
        (1, "active"),
        (2, "inactive"),
        (3, "active"),
    ], ["id", "status"])
    
    # Expected output
    expected_df = spark.createDataFrame([
        (1, "active"),
        (3, "active"),
    ], ["id", "status"])
    
    # Test transformation
    result_df = filter_active_status(input_df)
    
    # Assert equality
    assert_df_equality(result_df, expected_df)

def test_enrich_with_age_group(spark: SparkSession):
    """Test age group enrichment"""
    input_df = spark.createDataFrame([
        (1, 15),
        (2, 25),
        (3, 45),
        (4, 65),
    ], ["id", "age"])
    
    result_df = enrich_with_age_group(input_df)
    
    age_groups = [row.age_group for row in result_df.collect()]
    assert age_groups == ["Under 18", "18-34", "35-54", "55+"]
```

### Configuration-Driven Table Definitions

```python
from dataclasses import dataclass
from typing import List, Dict

@dataclass(frozen=True)
class TableConfig:
    """Immutable configuration for table definitions"""
    name: str
    source_table: str
    partition_cols: List[str]
    table_properties: Dict[str, str]
    comment: str

# Configuration instances
CUSTOMER_CONFIG = TableConfig(
    name="customers_silver",
    source_table="bronze.customers",
    partition_cols=["country"],
    table_properties={
        "delta.autoOptimize.optimizeWrite": "true",
        "delta.targetFileSize": "128mb"
    },
    comment="Silver layer customer data"
)

# Use configuration in table definition
@dp.table(
    name=CUSTOMER_CONFIG.name,
    partition_cols=CUSTOMER_CONFIG.partition_cols,
    table_properties=CUSTOMER_CONFIG.table_properties,
    comment=CUSTOMER_CONFIG.comment
)
def customers_silver():
    return spark.table(CUSTOMER_CONFIG.source_table).filter(...)
```

## Summary

In this notebook, we explored how to define tables, views, and sinks in Lakeflow Declarative Pipelines:

### Key Concepts Covered

1. **Batch Tables (`@dp.table`)**
   - Materialized Delta tables for primary data storage
   - Configuration options: partitioning, clustering, table properties
   - Pure function definitions (no side effects)

2. **Materialized Views (`@dp.materialized_view`)**
   - Pre-computed aggregations and complex joins
   - Refreshed on each pipeline run
   - Ideal for analytics and reporting workloads

3. **Temporary Views (`@dp.temporary_view`)**
   - Logical views without physical storage
   - Best for intermediate transformations
   - Reusable subquery patterns

4. **Table Properties**
   - Delta Lake optimization settings
   - Partitioning strategies
   - Liquid clustering (Databricks-specific)

5. **Dependency Management**
   - Using `dp.read()` for pipeline dependencies
   - Automatic dependency resolution
   - Complex multi-layer architectures

6. **Custom Sinks**
   - Exporting to external systems
   - JDBC, cloud storage, messaging systems
   - Complementing Delta Lake tables

7. **Testability**
   - Pure transformation functions
   - Functional composition with `.transform()`
   - Configuration-driven definitions

### Functional Programming Benefits

- **Pure Functions**: Table definitions return DataFrames without side effects
- **Composition**: Build complex pipelines from simple, testable functions
- **Immutability**: Tables are immutable, transformations create new tables
- **Declarative**: Focus on what tables contain, not how to build them

### Best Practices

✅ Keep table functions pure (no `.write()`, `.collect()`, `.count()`)
✅ Use appropriate decorator for use case (table vs view vs temporary view)
✅ Extract transformation logic into testable pure functions
✅ Configure table properties for optimal performance
✅ Use temporary views for cheap intermediate steps
✅ Materialize expensive computations used multiple times

### Next Steps

- **6.3**: Streaming tables and real-time processing
- **6.4**: Data quality with expectations
- **6.5**: Advanced flows and CDC patterns
- **6.6**: Best practices and anti-patterns


## Exercises

Practice defining tables and views:

**Exercise 1: Table Type Selection**
For each scenario, choose `@dp.table`, `@dp.materialized_view`, or `@dp.temporary_view`:
- Product catalog updated nightly
- Complex 4-table join for dashboard
- Filter step: "orders from last 30 days"
- Daily sales aggregation by region

**Exercise 2: Pure Function Extraction**
- Take an existing table definition
- Extract transformation logic into pure functions
- Compose using `.transform()`
- Write unit tests for each function

**Exercise 3: Configuration Design**
- Create a `TableConfig` dataclass for your domain
- Define table properties for optimal performance
- Choose appropriate partitioning strategy
- Document configuration decisions

**Exercise 4: Dependency Graph**
- Design a 3-layer pipeline (bronze/silver/gold)
- Define 5-7 tables with dependencies
- Include temporary views for reusable logic
- Add materialized view for complex aggregation

**Exercise 5: Sink Configuration**
- Identify a table that should be exported
- Design sink configuration for external system
- Choose appropriate export format and mode
- Document integration requirements
